In [50]:
import pandas as pd

#Import the data from 2 files (My computer runs out of memory with more)

#alldata = pd.read_csv('./dataset/AllData.csv', low_memory=False)
data_part1 = pd.read_csv('./dataset/Data_part1.csv', low_memory=False)
data_part1Stage2 = pd.read_csv('./dataset/Data_part1Stage2.csv', low_memory=False)
#data_part2 = pd.read_csv('./dataset/Data_part2.csv', low_memory=False)
#data_part2Stage2 = pd.read_csv('./dataset/Data_part2Stage2.csv', low_memory=False)
#data_part3 = pd.read_csv('./dataset/Data_part3.csv', low_memory=False)
#data_part3Stage2 = pd.read_csv('./dataset/Data_part3Stage2.csv', low_memory=False)
#data_part4 = pd.read_csv('./dataset/Data_part4.csv', low_memory=False)
#data_part4Stage2 = pd.read_csv('./dataset/Data_part4Stage2.csv', low_memory=False)
#data_part5Stage2 = pd.read_csv('./dataset/Data_part5Stage2.csv', low_memory=False)

In [51]:
#all_dataset_names = [alldata, data_part1, data_part1Stage2, data_part2, data_part2Stage2, data_part3, data_part3Stage2, data_part4, data_part4Stage2, data_part5Stage2]
dataset_names = [data_part1, data_part1Stage2] #Files which are used
complete_dataset =  pd.concat(dataset_names, ignore_index=True, sort=False).drop(columns=['Unnamed: 0']) #Put the files into one dataframe
complete_dataset = complete_dataset.drop(columns=list(complete_dataset.filter(regex = '^cf'))) #Remove all columns with cf
complete_dataset = complete_dataset.set_index('id') #Create an explicit index. Each row can now be refered to by it's ID

In [52]:
#Get all of the rows which depend on something
index_dataset = complete_dataset[['depends_on']]
index_dataset = index_dataset.dropna()

#If you have multiple values in one cell, split them into different columns (Ex: 2 items seperated by a comma = 2 columns)
targets = index_dataset['depends_on'].str.split(',', expand=True)
#Count the number of instances in each row
result = targets.apply(pd.Series.value_counts)
#Remove NaN values - they throw things off
result = result.fillna(0)

#Sum all the columns, based on row. This will give us the total count for how many entries 
#are dependent on another, with the index being the ID of the "depends_on" entry
result = result.sum(axis='columns').to_frame()

#Find the entries which have the most references in the "depends_on" column
result = result.sort_values(by = 0, ascending=False)

# Give me the top 150 referenced entries
result = result.head(150)

#Make a list of the entries
indicies = result.index.values.tolist()

#Make sure the entries are in string format, with all whitespace stripped
indicies = list(map(str,indicies))
indicies = [i.strip() for i in indicies]
print(indicies)

['1395112', '1130266', '1529362', '1445197', '1448703', '1575284', '1574241', '278458', '1470432', '1231208', '1471622', '1444991', '1319773', '1172897', '1464426', '1472491', '1320475', '1461360', '1574159', '1460343', '1477783', '231429', '1436478', '1511073', '1488620', '1459209', '1317102', '1322630', '1457891', '1496671', '1549775', '1452845', '1478118', '1444489', '1205323', '146679', '1461793', '1350424', '1446830', '1485081', '1408053', '1450956', '1217996', '1344091', '106592', '1218018', '1062849', '1464311', '1457988', '1457891', '1459555', '1460805', '260839', '132035', '1424281', '1574159', '1356397', '1483483', '1427928', '1446101', '1473772', '125419', '1353360', '1512802', '1248897', '1480504', '1464828', '1444489', '1475189', '1402247', '1467221', '1477676', '910412', '1471625', '231062', '57805', '1478372', '675539', '1466549', '1439588', '1467949', '1342051', '1482682', '1456680', '17483', '1191460', '1476876', '1473921', '1388019', '1446168', '1441308', '1418995', '

In [53]:
#Import the library which has the stopwords (if, the, etc...)
import nltk
nltk.download('stopwords')
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords

#Give me the depends_on and summary column
complete_dataset = complete_dataset[['depends_on', 'summary']]
print(complete_dataset.shape) #Check how big the dataframe is

#Used to remove the stopwords. I had to make a funciton because it wouldn't work any other way
def removeStopWords(startingList):
    returnedList = [x for x in startingList if x not in stopwords.words('english')]
    return returnedList

#Used to remove punctuation from the summary column
tokenizer = RegexpTokenizer(r'\w+')

#Give me the summary column
summary_features = complete_dataset['summary']
summary_features = summary_features.to_frame()
#Remove punctuation
summary_features = summary_features.apply(lambda x: tokenizer.tokenize(x['summary']), axis=1)
#Remove stopwords
summary_features = summary_features.apply(lambda x: removeStopWords(x))

#Get the "depends_on" column into its own dataframe, change each cell to hold a list of strings (each ID = 1 string) instead of one string
targets = complete_dataset['depends_on'].str.split(',').to_frame()
#Replace all NaN values with empty lists
targets['depends_on'] = targets['depends_on'].apply(lambda x: x if isinstance(x, list) else [])
summary_features = summary_features.to_frame()
targets

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Madis\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


(14417, 2)


,depends_on
id,
1572867,[]
1572870,[]
1572872,[]
1441804,[1455351]
1048589,[]
...,...
1474961,"[1450447, 1476315, 1476596]"
1474968,[1475252]
1212826,[]


In [54]:
from sklearn.preprocessing import MultiLabelBinarizer

#Some code taken and modified from https://stackoverflow.com/questions/41150846/one-hot-encoding

#Change the summary dataframe to onehot encoding - each row can have more than one "1" because sentences have multiple words
mlb = MultiLabelBinarizer()
features = summary_features.join(pd.DataFrame(mlb.fit_transform(summary_features.pop(0)),columns=mlb.classes_,index=summary_features.index))

#Change the depends_on dataframe to onehot encoding - each row can have more than one "1" because you can have more than one dependency
targets = targets.join(pd.DataFrame(mlb.fit_transform(targets.pop('depends_on')),columns=mlb.classes_,index=targets.index))

#Get rid of whitespace in the column names
targets.columns = targets.columns.str.strip()
targets

,1000077,1000199,1000317,1000370,1000462,1000745,1000775,1000814,1000870,1000879,...,992569,9942,995252,996174,997190,997779,997906,998343,998494,999754
id,,,,,,,,,,,,,,,,,,,,,
915,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2654,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2678,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2800,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2875,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1575726,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1575749,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1575756,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [55]:
#Save the list of all target values
all_indicies = targets.index.values.tolist()
full_target_list = targets

In [56]:
#Select only the columns which have entries in the top 150 dependencies
targets = targets[indicies]
#Get rid of duplicate rows and columns
targets = targets.groupby(level=0, axis=1).sum()
targets = targets.groupby(level=0, axis=0).sum()
#Get rid of rows which no longer have any dependencies (ones that depended on nothing in the top 40)
targets = targets.loc[(targets != 0).any(axis=1)]
targets

,103843,104960,1062849,106592,1073717,1130266,1172897,1191460,1205323,1217996,...,231429,252848,260839,278458,453908,547004,57805,672244,675539,910412
id,,,,,,,,,,,,,,,,,,,,,
10097,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
16903,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
16913,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19403,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
29106,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1575280,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1575281,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1575282,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [57]:
#Select twice as many bugs with no dependencies as there were bugs with at least 1 dependency.
from random import choices

non_dependent_indicies = [i for i in all_indicies if i not in targets.index.values.tolist()]

non_dependent_indicies = choices(non_dependent_indicies, k=(len(targets.index.values.tolist())*2))

#Make a list of all the targets we want to use (with and without dependent bugs)
all_indicies_final = non_dependent_indicies + targets.index.values.tolist()

print(len(all_indicies_final))

1674


In [58]:
#Retrieve the full list of depends_on bugs we want to use

targets = full_target_list.loc[all_indicies_final]
targets = targets[indicies]

targets

,1395112,1395112,1130266,1529362,1445197,1445197,1448703,1575284,1574241,1574241,...,1314079,1411707,1452645,1452645,1455462,1339683,1431246,1489392,1473601,1473601
id,,,,,,,,,,,,,,,,,,,,,
1445794,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
281354,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
281354,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
281354,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
281354,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1575283,0,0,0,0,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1575283,0,0,0,0,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1575283,0,0,0,0,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0


In [59]:
#Get rid of duplicate rows and columns (in the new dataframe)
targets = targets.groupby(level=0, axis=1).sum()
targets = targets.groupby(level=0, axis=0).sum()
targets


,103843,104960,1062849,106592,1073717,1130266,1172897,1191460,1205323,1217996,...,231429,252848,260839,278458,453908,547004,57805,672244,675539,910412
id,,,,,,,,,,,,,,,,,,,,,
2892,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8345,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10097,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10713,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10791,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1575490,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1575507,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1575579,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [60]:
#Get the indicies of the selected target values
target_indicies = targets.index.values.tolist()

#Remove duplicate rows
features = features.groupby(level=0, axis=0).sum()
#Select features which match the target indicies
features = features.loc[target_indicies]
#Remove columns which are all 0s
features = features.loc[:, features.any()]

features

,0,002,008,01,010,011,092ada1154b72fe71d2f227a5df0239586d2323a,1,10,10000,...,ycbcr,yosemite,youtube,z,zero,zh,zombification,zoom,zoomed,zooming
id,,,,,,,,,,,,,,,,,,,,,
2892,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8345,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10097,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10713,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10791,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1575490,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1575507,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1575579,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [61]:
#Make sure all values are either 1 or 0
def set_one_or_zero(x):
    if x == 0 or x == 1:
        return
    else:
        return (x/x)


targets.applymap(set_one_or_zero)
features.applymap(set_one_or_zero)

targets

,103843,104960,1062849,106592,1073717,1130266,1172897,1191460,1205323,1217996,...,231429,252848,260839,278458,453908,547004,57805,672244,675539,910412
id,,,,,,,,,,,,,,,,,,,,,
2892,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8345,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10097,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10713,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10791,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1575490,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1575507,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1575579,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [62]:
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

#Some code taken (and modified) from https://www.askpython.com/python/examples/k-fold-cross-validation

kfold = KFold(n_splits=3, shuffle=True)

#Use random forest
random_forest = RandomForestClassifier()

#We can have multiple outputs - hence, we need to use a MultiOutputClassifier
output_classifier = MultiOutputClassifier(random_forest)

f1 = []
precision = []
recall = []


for train_index , test_index in kfold.split(features):
    X_train , X_test = features.iloc[train_index,:],features.iloc[test_index,:]
    y_train , y_test = targets.iloc[train_index,:] , targets.iloc[test_index,:]
     
    output_classifier.fit(X_train, y_train)
    y_predict = output_classifier.predict(X_test)

    #Changes our results of y_predict & y_tesy into a format which the f1_score function will accept
    binarizer = MultiLabelBinarizer().fit(y_test)

    Y_test = binarizer.fit_transform(y_test.values.tolist())
    Y_predict = binarizer.transform(y_predict)

    pred_values = output_classifier.predict(X_test)
     
    f1.append(f1_score(Y_test, Y_predict, average='weighted'))
    precision.append(precision_score(Y_test, Y_predict, average='weighted', zero_division=0))
    recall.append(recall_score(Y_test, Y_predict, average='weighted'))

print("F1-score: {}".format(sum(f1)/3))
print("Precision-score: {}".format(sum(precision)/3))
print("Recall-score: {}".format(sum(recall)/3))

F1-score: 0.8096980446188712
Precision-score: 0.9799984773354624
Recall-score: 0.781092532009542
